# Raku RAG demo

### ... and *Semantic Nearest Neighbors Graphs*

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)      
September 2024 


-----

## Introduction

The so called ***Retrieval Augmented Generation (RAG)*** functionalities of Large Language Models (LLMs) are demonstrated using semantic nearest neighbors graphs.

Vector Databases (VDBs) made with the Raku package ["LLM::RetrievalAugmentedGeneration"](https://raku.land/zef:antononcube/LLM::RetrievalAugmentedGeneration) are used.

1. Semantic graph for words 
    - Select a set of words
    - Find LLM embeddings for the those words
    - Find nearest neighbor graph for the obtained vectors

2. Nearest neighbor graph -- joined VDBs
    - Ingest VDBs corresponding to (long) podcast interviews
        - "Modern Wisdom" with [Eric Weinstein](https://en.wikipedia.org/wiki/Eric_Weinstein)
        - Eclectic and long interviews (≈3.5 hours each)
        - Heterogeneous content (wide variety of topics)
        - *(Hence, suitable for semantic similarities demos)*
    - Merge VDBs
    - Make a corresponding semantic graph
        - Nearest neighbor graph
        - Connected components

3. Nearest neighbor graph and RAG 
    - Create a semantic relationship graph for an interview
    - LLM-derivation of summaries for top connected components
    - Graph plot traces 
        - Using suitable graph tooltips
    - Do a retrieval based LLM summary for a query

-----

## Setup

In [2]:
use Data::Importers;
use LLM::Functions;
use XDG::BaseDirectory :terms;

use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use Data::Reshapers;
use Data::Summarizers;
use Math::Nearest;
use Math::DistanceFunctions::Native;
use Statistics::OutlierIdentifiers;

use NativeCall;

use Math::Nearest;
use Graph;
use JavaScript::D3;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [ ]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [ ]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [ ]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

-------

## Words semantic graph

```mermaid
flowchart LR
    WS[Words set selection] --> WE[Word embeddings]
    WE --> WNN
    WNN[Nearest neighbor graph<br>one neighbor] --> WCC[Connected component<br>to focus on]
    WCC --> WNN2[Nearest neighbor graph<br>two neighbors]
    WNN2 --> WGP[Graph plot]
    WE <-.-> LLMs
    
    subgraph WGraph[Words Graph]
        WS
        WE
        WNN
        WNN2
        WCC
        WGP
    end

    subgraph LLMs
        OpenAI{{OpenAI}}
        Gemini{{Gemini}}
        MistralAI{{MistralAI}}
        LLaMA{{LLaMA}}
    end
```

-------

## Words graph making

Here is a set of words:

In [ ]:
my @content = <angel apple ardvark bible car cat cherry plum chocolate cookie cow devil film horse house movie projector raccoon tiger tree>;

@content.elems

Here we specify an LLM-access configuration:

In [ ]:
my $conf = llm-configuration('Gemini');
$conf.Hash.elems

Here we create semantic search index:

In [ ]:
my $vdbObjSmall = create-semantic-search-index(@content, e => $conf, name => 'words')

Here we see the dimensions of the obtained vectors:

In [ ]:
$vdbObjSmall.vectors».elems

Here we find the embedding of a certain word (using the same LLM model as above):

In [ ]:
my $vec = llm-embedding("coffee", e => $conf).head;
$vec.elems

Here we find the closest Nearest Neighbors (NNs) of that word:

In [ ]:
my @nns = $vdbObjSmall.nearest($vec, 3, prop => 'label' ).map(*.Slip)

Here are the corresponding words:

In [ ]:
$vdbObjSmall.items{@nns}

Here we find the corresponding NNs graph with 1 and 2 nns per vertex:

In [ ]:
my ($gr1, $gr2) = [1, 2].map({ 
        # NNs graph edges
        my @edges = nearest-neighbor-graph(
            $vdbObjSmall.vectors.pairs, 
            $_, 
            method => 'Scan', 
            distance-function => &euclidean-distance, 
            format => 'dataset'
        );

        # Replace IDs with names
        @edges .= map({ $_<from> = $vdbObjSmall.items{$_<from>}; $_<to> = $vdbObjSmall.items{$_<to>}; $_ });
        
        # Make the graph
        Graph.new(@edges)
}).flat;

.say for ($gr1, $gr2)


Find 1-nns graph's connected components:

In [ ]:
my @comps = $gr1.connected-components.sort(-*.elems)

In [ ]:
#%js

$gr2.edges(:dataset)
==> js-d3-graph-plot(
        :$background,
        highlight => [|@comps.head, |$gr1.subgraph(@comps.head).edges],
        width => 600,
        vertex-label-color => 'Ivory',
        edge-thickness => 2,
        vertex-size => 3,
        vertex-color => 'Blue',
        edge-color => 'SteelBlue',
        force => { charge => {strength => -200, iterations => 4}, collision => {iterations => 1, radius => 10} }
    )

------

## Semantic nearest neighbors graph

```mermaid
flowchart LR
    subgraph LocalVDB[Local Folder]
        direction LR
        A(Vector Database 1)
        B(Vector Database 2)
        C(Vector Database N)
    end

    subgraph Creation
        ID
        SD
        EV
        CD
    end

    subgraph NNGraph[Semantic Graph]
        D 
        E
        CC
        T 
        GP   
    end

    ID[Ingest document collection]
    SD[Split Documents]
    EV[Get LLM Embedding Vectors]
    CD[Create Vector Database]
    ID --> SD --> EV --> CD

    EV <-.-> LLMs
    
    CD -.- CArray[[CArray<br>representation]]

    CD -.-> |export| LocalVDB

    LocalVDB -.- JSON[[JSON<br>representation]]

    LocalVDB -.-> |import|D[Ingest Vector Database]
 
    D -.- CArray
    D --> E
    E[Nearest neighbor graph] --> CC[Connected Components]
    E -.- |nearest neighbors<br>distance function|CArray
    CC --> T[LLM-derived titles<br>per component]
    T --> GP[Graph plot]

    subgraph LLMs
        OpenAI{{OpenAI}}
        Gemini{{Gemini}}
        MistralAI{{MistralAI}}
        LLaMA{{LLaMA}}
    end
```

------

## Ingest vector databases

Ingest vector database from the default directory and show a summary:

In [ ]:
#% html
my @field-names = <id name item-count dimension version llm-service llm-embedding-model created>;

vector-database-objects(f=>'hash', :flat)

==> { $_.map({ $_<created> = $_<file>.IO.created.DateTime.Str.subst('T',' ').substr(^19); $_}).sort(*<created>).reverse }()
==> to-html(:@field-names)

Get specific databases:

In [ ]:
#my @vdbs = vector-database-objects(f=>'hash', :flat).grep({ $_<name> ∈ <No833 No747> && $_<llm-service> eq 'gemini' }).map({ create-vector-database(file => $_<file>) })

my @vdbs = vector-database-objects(f=>'hash', :flat).grep({ $_<id> ∈ <266b20ca-d917-4ac0-9b0a-7c420625666c d2effebc-2cef-4b2b-84ca-5dcfa3c1864b> }).map({ create-vector-database(file => $_<file>) })

Sample vectors:

In [ ]:
.say for @vdbs.head.vectors.pick(3)

Sample items:

In [ ]:
#% html
@vdbs.head.items.pick(3).map({ <key value> Z=> $_.kv })».Hash.Array 
==> to-html(field-names => <key value>, align => 'left')

-----

## Combined databases graph

Here we join the selected databases:

In [ ]:
my $vdbObj2 = vector-database-join(@vdbs)

Find a nearest neighbors graph:

In [ ]:
my @edges2 = nearest-neighbor-graph($vdbObj2.vectors.pairs, 1, method => 'Scan', distance-function => &euclidean-distance, format => 'raku');

my $gr2 = Graph.new(@edges2)

In [ ]:
my @comps2 = $gr2.connected-components.sort(-*.elems).head(8);

@comps2.elems

In [ ]:
#% js
@edges2
==> js-d3-graph-plot(
        :$background,
        highlight => @comps2.map({[|$_, |$gr2.subgraph($_).edges]}),
        vertex-label-color => 'none',
        edge-thickness => 2,
        vertex-size => 4,
        vertex-color => 'Blue',
        width => 1100,
        height => 950,
        edge-color => 'Gray',
        vertex-color => 'DarkGray',
        #force => {charge => {strength => -3, iterations => 4}, collision => {radius => 14, iterations => 4}, link => {distance => 1};}
    )

-----

## Nearest neighbor graph

Take one of the vector databases:

In [ ]:
my $vdbObj = @vdbs.head

In [ ]:
.say for $vdbObj.items.pick(3)

Make the nearest neighbor graph for the vectors in the vector database:

In [ ]:
my @edges = nearest-neighbor-graph($vdbObj.vectors.pairs, 1, method => 'Scan', distance-function => &euclidean-distance, format => 'raku')

Make the graph:

In [ ]:
my $gr = Graph.new(@edges)

Get graph's connected components:

In [ ]:
my @comps = $gr.connected-components.sort(-*.elems);
.say for @comps.head(12)

Example paragraph:

In [ ]:
#% markdown

$vdbObj.items<170.0>

LLM function for naming a set of paragraphs:

In [ ]:
my &fNamer = llm-function({"Summarize the text into a very short sentence that has at most 8 words: \n\n $_"})

Example title finding:

In [ ]:
@comps[5]

In [ ]:
&fNamer( $vdbObj.items{|@comps[5]}.join("\n") )

Find titles for some of the largest components:

In [ ]:
my @titles = @comps.head(6).map({ &fNamer( $vdbObj.items{|$_}.join("\n") ) });

@titles.elems

In [ ]:
.say for @titles

Make rules for all components and titles:

In [ ]:
my @res = do for ^@titles.elems -> $i {
    my @vals = @comps[$i].Array X~ ' : ' X~  @titles[$i];
    @comps[$i].Array Z=> @vals
}

my %rules = @res.map(*.Slip);

%rules.elems


Make graph highlight specification:

In [ ]:
my @colors = <#1f77b4 #ff7f0e #2ca02c #d62728 #9467bd #8c564b #e377c2 #7f7f7f #bcbd22 #17becf>;

my $highlight = (@colors.head(6).Array Z=> @comps.head(6).map({ [ |$_.map({ %rules{$_} // $_ }), |$gr.subgraph($_).edges.map({ ( %rules{$_.key} // $vdbObj.items{$_.key} ) => ( %rules{$_.value} // $vdbObj.items{$_.value} ) }) ]})).Hash;

Semantic graph:

In [67]:
#%js
my @edges2 = @edges.map({ ( %rules{$_.key} // $vdbObj.items{$_.key} ) => ( %rules{$_.value} // $vdbObj.items{$_.value} ) });

@edges2
==> js-d3-graph-plot(
        :$background,
        :$highlight,
        vertex-label-color => 'none',
        edge-thickness => 2,
        vertex-size => 4,
        vertex-color => 'Blue',
        width => 1100,
        height => 800,
        edge-color => 'Gray',
        vertex-color => 'DarkGray',
    )

Generate LLM based on one of components:

In [ ]:
#% markdown

llm-synthesize([ 
    "Summarize the following text into a list of three-four points, each with no more than 8 words:",
    $vdbObj.items{|@comps[5].sort}.join(" "),
], e => $conf)

-------

## References

### Articles

[AA1] Anton Antonov, 
["Outlier detection in a list of numbers"](https://rakuforprediction.wordpress.com/2022/05/29/outlier-detection-in-a-list-of-numbers/),
(2022),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::PaLM Raku package](https://github.com/antononcube/Raku-WWW-PaLM),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp5] Anton Antonov,
[ML::FindTextualAnswer Raku package](https://github.com/antononcube/Raku-ML-FindTextualAnswer),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp6] Anton Antonov,
[Math::Nearest Raku package](https://github.com/antononcube/Raku-Math-Nearest),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp7] Anton Antonov,
[Math::DistanceFunctions Raku package](https://github.com/antononcube/Raku-Math-DistanceFunctions),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp8] Anton Antonov,
[Statistics::OutlierIdentifiers Raku package](https://github.com/antononcube/Raku-Statistics-OutlierIdentifiers),
(2022),
[GitHub/antononcube](https://github.com/antononcube).

## Videos

[AAv1] Anton Antonov,
["Raku RAG demo"](https://www.youtube.com/watch?v=JHO2Wk1b-Og),
(2024),
[YouTube/AAA4prediction](https://www.youtube.com/@AAA4prediction).

[CWv1] Chris Williamson,
["Eric Weinstein - Why Does The Modern World Make No Sense? (4K)"](https://www.youtube.com/watch?v=p_swB_KS8Hw),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/747-eric-weinstein-why-does-the-modern-world-make-no-sense).)

[CWv2] Chris Williamson,
["Eric Weinstein - Are We On The Brink Of A Revolution? (4K)"](https://www.youtube.com/watch?v=PYRYXhU4kxM),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution).)